In [1]:
from src.utils import extract_feature_from_image, postprocessing
from ultralytics import YOLO
from pycaret.classification import load_model, predict_model
import easyocr
import pandas as pd

In [4]:
# define image and model path
image_dir = "sample_images/"
filename = "1.jpg"
box_model_path = "models/yolov8_box.pt"
symbol_model_path = "models/yolov8_symbol.pt"
classifier_path = "models/extract_model"

# load model
box_model = YOLO(box_model_path)
symbol_model = YOLO(symbol_model_path)
classifier = load_model(classifier_path)
reader = easyocr.Reader(["en"], gpu=True)

Transformation Pipeline and Model Successfully Loaded


In [5]:
# extract data from image
extract_rt_df, extract_lt_df = extract_feature_from_image(
    filename, image_dir, box_model, symbol_model, reader
)

# post processing right and left data
feature_rt_df = postprocessing(extract_rt_df)
feature_lt_df = postprocessing(extract_lt_df)


0: 640x448 1 ear_graph, 1 pta_table, 12.1ms
Speed: 2.0ms preprocess, 12.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 7 Air Rt Unmaskeds, 8 Air Lt Unmaskeds, 1 Air Rt Masked, 4 Bone Lt Unmaskeds, 4 Bone Rt Maskeds, 12.2ms
Speed: 2.0ms preprocess, 12.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


In [6]:
# classified degree of hearing loss
predict_right_hearing_loss = predict_model(classifier, data=feature_rt_df)
predict_left_hearing_loss = predict_model(classifier, data=feature_lt_df)

# concat right and left data and rename index
result = pd.concat([predict_right_hearing_loss, predict_left_hearing_loss], axis=0)
result.index = ['right', 'left']

# map prediction label
label = ['Normal', 'Mild', 'Moderate', 'Moderately Severe', 'Severe', 'Profound']
result['prediction_label'] = result['prediction_label'].apply(lambda x: label[x])

result

,pta_ac,pta_bc,sl,srt,pb,250_ac,500_ac,1000_ac,2000_ac,4000_ac,6000_ac,8000_ac,500_bc,1000_bc,2000_bc,4000_bc,prediction_label,prediction_score
right,59.646687,59.082153,57.041077,55.0,6.0,30.950430,36.472107,40.953014,37.195522,41.783997,41.590408,56.022800,35.959709,40.815903,35.442680,41.792877,Moderately Severe,1.0
left,38.206882,37.406097,30.000000,40.0,4.0,76.198174,60.909603,56.373283,61.657188,55.976532,65.993835,71.006042,60.096394,56.103058,61.047016,55.750866,Mild,1.0
